# Textual Entailment

In [1]:
!nvidia-smi
!lscpu

Wed Aug  4 12:54:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0
!pip install sentence-transformers==2.0.0

     |████████████████████████████████| 316 kB 6.4 MB/s 
     |████████████████████████████████| 233 kB 11.4 MB/s 
     |████████████████████████████████| 1.4 MB 12.4 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=be03432ed873e210dbc92e0b23c8f7c279d925ff2c5de81aad36bc3047ae9aed
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154223 sha256=a30a4d8b4862f3e43ad2e2542794886191449e891abb00ca7103b8ec984b5cd6
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 43 kB 1.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.

In [3]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Import required packages
import re
import gc
import os
import hazm
import time
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import MT5Config, MT5ForConditionalGeneration, MT5Tokenizer

from sentence_transformers import models, SentenceTransformer, util

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
class TextualEntailmentDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Textual Entailment. """

    def __init__(self, premises_hypotheses, targets, label_list, tokenizer, model_type, max_length):
        self.premises_hypotheses = premises_hypotheses
        self.tokenizer = tokenizer
        self.model_type = model_type
        self.targets = targets
        self.max_length = max_length
        self.label2index = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
        self.index2label = {i: label for label, i in self.label2index.items()}

    def __len__(self):
        return len(self.premises_hypotheses)

    def __getitem__(self, item):
        encoding = self.tokenizer(
            self.premises_hypotheses[item],
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        if self.model_type == "mt5":
            inputs = {
                'premise': self.premises_hypotheses[item].split('<sep>')[0],
                'hypothesis': self.premises_hypotheses[item].split('<sep>')[1],
                'pair': self.premises_hypotheses[item],
                'targets': torch.tensor(self.label2index[self.targets[item]], dtype=torch.long),
                'original_targets': self.targets[item],
                'input_ids': encoding.input_ids.flatten(),
                'attention_mask': encoding.attention_mask.flatten()
            }
        else:
            inputs = {
                'premise': self.premises_hypotheses[item].split('<sep>')[0],
                'hypothesis': self.premises_hypotheses[item].split('<sep>')[1],
                'pair': self.premises_hypotheses[item],
                'targets': torch.tensor(self.label2index[self.targets[item]], dtype=torch.long),
                'original_targets': self.targets[item],
                'input_ids': encoding.input_ids.flatten(),
                'attention_mask': encoding.attention_mask.flatten(),
                'token_type_ids': encoding['token_type_ids'].flatten()
            }
        return inputs


class TextualEntailmentDataset2(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Textual Entailment. """

    def __init__(self, premises, hypotheses, targets, label_list, tokenizer, model_type, max_length):
        self.premises = premises
        self.hypotheses = hypotheses
        self.tokenizer = tokenizer
        self.model_type = model_type
        self.targets = targets
        self.max_length = max_length
        self.label2index = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
        self.index2label = {i: label for label, i in self.label2index.items()}

    def __len__(self):
        return len(self.premises)

    def __getitem__(self, item):
        encoding = self.tokenizer(
            [(self.premises[item], self.hypotheses[item])],
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        if self.model_type == "mt5":
            inputs = {
                'premise': self.premises[item],
                'hypothesis': self.hypotheses[item],
                'targets': torch.tensor(self.label2index[self.targets[item]], dtype=torch.long),
                'original_targets': self.targets[item],
                'input_ids': encoding.input_ids.flatten(),
                'attention_mask': encoding.attention_mask.flatten()
            }
        else:
            inputs = {
                'premise': self.premises[item],
                'hypothesis': self.hypotheses[item],
                'targets': torch.tensor(self.label2index[self.targets[item]], dtype=torch.long),
                'original_targets': self.targets[item],
                'input_ids': encoding.input_ids.flatten(),
                'attention_mask': encoding.attention_mask.flatten(),
                'token_type_ids': encoding['token_type_ids'].flatten()
            }
        return inputs


class TextualEntailmentSimilarityDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Textual Entailment. """

    def __init__(self, premises, hypotheses, targets):
        self.premises = premises
        self.hypotheses = hypotheses
        self.targets = targets

    def __len__(self):
        return len(self.premises)

    def __getitem__(self, item):
        inputs = {
            'item': item,
            'premise': self.premises[item],
            'hypothesis': self.hypotheses[item],
            'targets': self.targets[item]
        }
        return inputs


class TextualEntailment:
    def __init__(self, model_name, model_type, label_list):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        self.model_type = model_type.lower()
        self.label_list = label_list
        if self.model_type == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(model_name)
            self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
            self.config = MT5Config.from_pretrained(self.model_name)
        elif self.model_type == "sentence-transformer":
            word_embedding_model = models.Transformer(model_name)
            pooling_model = models.Pooling(
                word_embedding_model.get_word_embedding_dimension(),
                pooling_mode_mean_tokens=True,
                pooling_mode_cls_token=False,
                pooling_mode_max_tokens=False
            )
            self.model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
            self.id2label = self.config.id2label
            self.label2id = self.config.label2id

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() in ["parsinlu-natural", "parsinlu-mnli", "parsinlu-farstail"]:
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t", names=['premise_hypothesis', 'label'], header=None)

            # cleaning labels
            valid_labels = ['e', 'n', 'c']
            data['label'] = data['label'].apply(lambda r: r if r in valid_labels else None)
            data = data.dropna(subset=['label'])
            data = data.reset_index(drop=True)

            if 'label_map' in kwargs:
                data['label'] = data['label'].apply(lambda l: kwargs['label_map'][l])

            premise_hypothesis, labels = data['premise_hypothesis'].values.tolist(), data['label'].values.tolist()
            print(f'test part:\n #premise_hypothesis: {len(premise_hypothesis)}, #label: {len(labels)}')
            return premise_hypothesis, labels
        if dataset_name.lower() == "farstail":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, sep='\t')

            if 'label_map' in kwargs:
                data['label'] = data['label'].apply(lambda l: kwargs['label_map'][l])

            premises = data['premise'].values.tolist()
            hypotheses = data['hypothesis'].values.tolist()
            labels = data['label'].values.tolist()
            print(f'test part:\n #premise: {len(premises)}, #hypothesis: {len(hypotheses)}, #label: {len(labels)}')
            return premises, hypotheses, labels

    def textual_entailment_inference(self, premises, hypotheses, device, max_length):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return

        new_input = []
        for p, h in zip(premises, hypotheses):
            new_input.append((p, h))

        tokenized_batch = self.tokenizer(
            new_input,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        tokenized_batch = tokenized_batch.to(device)
        outputs = self.model(**tokenized_batch)
        pt_predictions = torch.argmax(F.softmax(outputs.logits, dim=1), dim=1)

        output_predictions = []
        for i, premise in enumerate(premises):
            output_predictions.append(
                (premise, hypotheses[i], pt_predictions[i].item(), self.label_list[pt_predictions[i].item()])
            )
        return output_predictions

    def mt5_textual_entailment_inference(self, premises, hypotheses, device):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        new_input = []
        for p, h in zip(premises, hypotheses):
            new_input.append(p + "<sep>" + h)

        tokenized_batch = self.tokenizer(
            new_input,
            padding=True,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        input_ids = tokenized_batch.input_ids.to(device)
        attention_mask = tokenized_batch.attention_mask.to(device)
        outputs = self.model.generate(input_ids=input_ids,
                                      attention_mask=attention_mask)
        predictions = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return predictions

    def sentence_transformer_textual_entailment_inference(self, premises, hypotheses, device, label_map=None):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if label_map is None:
            label_map = {"0<=score<0.4": "contradiction", "0.4<=score<=0.6": "neutral", "0.6<score<=1": "entailment"}
            print(f"Setting default value for label map: {label_map}")
        elif not all('score' in cond for cond in label_map.keys()):
            print(f"All the key of label_map must contain a condition on `score` variable.\n"
                  f"For example: label_map ={label_map}")
            return

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        premises_embeddings = self.model.encode(premises, convert_to_tensor=True, show_progress_bar=True)
        hypotheses_embeddings = self.model.encode(hypotheses, convert_to_tensor=True, show_progress_bar=True)

        # Compute the pair-wise cosine similarities
        similarity_scores, predicted_labels = [], []
        for i in range(len(premises)):
            cosine_score = \
                util.pytorch_cos_sim(premises_embeddings[i], hypotheses_embeddings[i]).cpu().detach().numpy()[0][0]
            similarity_scores.append(cosine_score)
            predicted_label = ''
            for exp in label_map:
                if eval(exp.replace('score', str(cosine_score))):
                    predicted_label = label_map[exp]
                    break
            predicted_labels.append(predicted_label)

        output_predictions = []
        for i, premise in enumerate(premises):
            output_predictions.append(
                (premise, hypotheses[i], similarity_scores[i], predicted_labels[i])
            )
        return output_predictions

    def evaluation(self, premise_hypothesis, labels, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return
        label_count = {label: labels.count(label) for label in labels}
        print("label_count:", label_count)
        dataset = TextualEntailmentDataset(premises_hypotheses=premise_hypothesis, targets=labels,
                                           label_list=self.label_list, tokenizer=self.tokenizer,
                                           model_type=self.model_type, max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#premise_hypothesis:{len(premise_hypothesis)}, #labels:{len(labels)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_loss, total_time = 0, 0
        output_predictions = []
        golden_labels, predicted_labels = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_premises = batch['premise']
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']
            b_token_type_ids = batch['token_type_ids']
            b_targets = batch['targets']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)
            b_token_type_ids = b_token_type_ids.to(device)
            b_targets = b_targets.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                                       token_type_ids=b_token_type_ids, labels=b_targets)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')
            # get the loss
            total_loss += b_outputs.loss.item()

            b_original_targets = batch['original_targets']
            golden_labels.extend(b_original_targets)

            b_predictions = torch.argmax(F.softmax(b_outputs.logits, dim=1), dim=1)
            b_predictions = b_predictions.cpu().detach().numpy().tolist()
            b_predictions = [dataset.index2label[label] for label in b_predictions]
            predicted_labels.extend(b_predictions)

            for i, premise in enumerate(b_premises):
                output_predictions.append((
                    premise,
                    batch['hypothesis'][i],
                    b_original_targets[i],
                    b_predictions[i]
                ))

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(data_loader)
        print("average loss:", avg_train_loss)
        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(premise_hypothesis))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(classification_report(
            golden_labels, predicted_labels, digits=10)))
        return output_predictions

    def evaluation_2(self, premises, hypotheses, labels, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return
        label_count = {label: labels.count(label) for label in labels}
        print("label_count:", label_count)
        dataset = TextualEntailmentDataset2(premises=premises, hypotheses=hypotheses, targets=labels,
                                            label_list=self.label_list, tokenizer=self.tokenizer,
                                            model_type=self.model_type, max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#premises:{len(premises)}, #hypotheses:{len(hypotheses)}, #labels:{len(labels)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_loss, total_time = 0, 0
        output_predictions = []
        golden_labels, predicted_labels = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_premises = batch['premise']
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']
            b_token_type_ids = batch['token_type_ids']
            b_targets = batch['targets']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)
            b_token_type_ids = b_token_type_ids.to(device)
            b_targets = b_targets.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                                       token_type_ids=b_token_type_ids, labels=b_targets)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')
            # get the loss
            total_loss += b_outputs.loss.item()

            b_original_targets = batch['original_targets']
            golden_labels.extend(b_original_targets)

            b_predictions = torch.argmax(F.softmax(b_outputs.logits, dim=1), dim=1)
            b_predictions = b_predictions.cpu().detach().numpy().tolist()
            b_predictions = [dataset.index2label[label] for label in b_predictions]
            predicted_labels.extend(b_predictions)

            for i, premise in enumerate(b_premises):
                output_predictions.append((
                    premise,
                    batch['hypothesis'][i],
                    b_original_targets[i],
                    b_predictions[i]
                ))

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(data_loader)
        print("average loss:", avg_train_loss)
        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(premises))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(classification_report(
            golden_labels, predicted_labels, digits=10)))
        return output_predictions

    def mt5_evaluation(self, premise_hypothesis, labels, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(premise_hypothesis) != len(labels):
            print('length of inputs and labels is not equal!!')
            return

        dataset = TextualEntailmentDataset(premises_hypotheses=premise_hypothesis, targets=labels,
                                           label_list=self.label_list, tokenizer=self.tokenizer,
                                           model_type=self.model_type, max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#premise_hypothesis:{len(premise_hypothesis)}, #labels:{len(labels)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_time = 0
        output_predictions = []
        golden_labels, predicted_labels = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_premises = batch['premise']
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model.generate(input_ids=b_input_ids, attention_mask=b_attention_mask)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')

            b_original_targets = batch['original_targets']
            golden_labels.extend(b_original_targets)

            b_predictions = self.tokenizer.batch_decode(b_outputs, skip_special_tokens=True)
            predicted_labels.extend(b_predictions)

            for i, premise in enumerate(b_premises):
                output_predictions.append((
                    premise,
                    batch['hypothesis'][i],
                    b_original_targets[i],
                    b_predictions[i]
                ))

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(premise_hypothesis))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(
            classification_report(golden_labels, predicted_labels, digits=10)))
        return output_predictions

    def evaluation_pair_similarity(self, premises, hypotheses, labels, device, label_map=None, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if label_map is None:
            label_map = {"0<=score<0.4": "contradiction", "0.4<=score<=0.6": "neutral",
                         "0.6<score<=1": "entailment"}
            print(f"Setting default value for label map: {label_map}")
        elif not all('score' in cond for cond in label_map.keys()):
            print(f"All the key of label_map must contain a condition on `score` variable.\n"
                  f"For example: label_map ={label_map}")
            return

        label_count = {label: labels.count(label) for label in labels}
        print("label_count:", label_count)
        print(f'#premises:{len(premises)}, #hypotheses:{len(hypotheses)}, #labels:{len(labels)}')

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_time = 0
        print("Start to evaluate test data ...")

        # Compute the sentence embeddings
        start = time.monotonic()
        premises_embeddings = self.model.encode(premises, convert_to_tensor=True, show_progress_bar=True,
                                                batch_size=batch_size)
        hypotheses_embeddings = self.model.encode(hypotheses, convert_to_tensor=True, show_progress_bar=True,
                                                  batch_size=batch_size)
        end = time.monotonic()
        print(f'time for computing embeddings of premises and hypotheses: {end - start}')
        total_time += end - start

        dataset = TextualEntailmentSimilarityDataset(premises=premises_embeddings, hypotheses=hypotheses_embeddings,
                                                     targets=labels)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print("#batch:", len(data_loader))

        output_predictions = []
        golden_labels, predicted_labels = [], []
        for step, batch in enumerate(data_loader):
            b_premises = batch['premise']
            b_hypotheses = batch['hypothesis']

            # move tensors to GPU if CUDA is available
            b_premises = b_premises.to(device)
            b_hypotheses = b_hypotheses.to(device)

            # Compute the pair-wise cosine similarities
            start = time.monotonic()
            cos_similarity_scores, b_predictions = [], []
            for i in range(len(b_premises)):
                cosine_score = util.pytorch_cos_sim(b_premises[i], b_hypotheses[i]).cpu().detach().numpy()[0][0]
                cos_similarity_scores.append(cosine_score)
                predicted_label = ''
                for exp in label_map:
                    if eval(exp.replace('score', str(cosine_score))):
                        predicted_label = label_map[exp]
                        break
                b_predictions.append(predicted_label)

            end = time.monotonic()
            total_time += end - start
            print(f'time for calculating cosine similarity in step {step}: {end - start}')

            golden_labels.extend(batch['targets'])
            predicted_labels.extend(b_predictions)

            for i, item in enumerate(batch['item']):
                output_predictions.append((
                    premises[item],
                    hypotheses[item],
                    cos_similarity_scores[i],
                    batch['targets'][i],
                    b_predictions[i]
                ))

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(premises))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(classification_report(
            golden_labels, predicted_labels, digits=10)))
        return output_predictions


In [6]:
model_name='persiannlp/wikibert-base-parsinlu-entailment'
te_model = TextualEntailment(model_name=model_name, model_type="wikibert", label_list = ['e', 'c', 'n'])
print(te_model.config)


BertConfig {
  "_name_or_path": "/Users/danielk/ideaProjects/entailment_models/wikibert-base-fa-cased_batch_size=8_learning_rate=3e-5_learning_rate=3e-5_num_train_epoch=7",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "entailment",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.7.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 20101
}



## Sample Inference

In [7]:
premise_list = [
    "این مسابقات بین آوریل و دسامبر در هیپودروم ولیفندی در نزدیکی باکرکی ، ۱۵ کیلومتری (۹ مایل) غرب استانبول برگزار می شود.",
    "آیا کودکانی وجود دارند که نیاز به سرگرمی دارند؟",
    "ما به سفرهایی رفته ایم که در نهرهایی شنا کرده ایم"
]
hypothesis_list = [
    "در ولیفندی هیپودروم، مسابقاتی از آوریل تا دسامبر وجود دارد.",
    "هیچ کودکی هرگز نمی خواهد سرگرم شود.",
    "علاوه بر استحمام در نهرها ، ما به اسپا ها و سونا ها نیز رفته ایم."
]
te_model.textual_entailment_inference(premise_list, hypothesis_list, device, max_length=te_model.config.max_position_embeddings)

[('این مسابقات بین آوریل و دسامبر در هیپودروم ولیفندی در نزدیکی باکرکی ، ۱۵ کیلومتری (۹ مایل) غرب استانبول برگزار می شود.',
  'در ولیفندی هیپودروم، مسابقاتی از آوریل تا دسامبر وجود دارد.',
  0,
  'e'),
 ('آیا کودکانی وجود دارند که نیاز به سرگرمی دارند؟',
  'هیچ کودکی هرگز نمی خواهد سرگرم شود.',
  1,
  'c'),
 ('ما به سفرهایی رفته ایم که در نهرهایی شنا کرده ایم',
  'علاوه بر استحمام در نهرها ، ما به اسپا ها و سونا ها نیز رفته ایم.',
  2,
  'n')]

## ParsiNLU Dataset

In [8]:
!git clone https://github.com/persiannlp/parsinlu
!ls parsinlu
!ls parsinlu/data/entailment/merged_with_farstail

Cloning into 'parsinlu'...
remote: Enumerating objects: 1434, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 1434 (delta 110), reused 139 (delta 82), pack-reused 1252
Receiving objects: 100% (1434/1434), 27.81 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (913/913), done.
data  LICENSE  README.md  requirements.txt  scripts  src
dev.tsv  test_farstail.tsv  test_natural.tsv  test_translation.tsv  train.tsv


### parsinlu natural subset

In [9]:
test_natural, test_labels = te_model.load_dataset_test_file(dataset_name="parsinlu-natural", dataset_file="./parsinlu/data/entailment/merged_with_farstail/test_natural.tsv")

test part:
 #premise_hypothesis: 850, #label: 850


In [10]:
!nvidia-smi
!lscpu

Wed Aug  4 12:57:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W /  70W |   1576MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
evaluation_output = te_model.evaluation(test_natural, test_labels, device, max_length=te_model.config.max_position_embeddings, batch_size=128)

label_count: {'c': 257, 'n': 274, 'e': 319}
#premise_hypothesis:850, #labels:850
#batch: 7
Start to evaluate test data ...
inference time for step 0: 0.020525259999999435
inference time for step 1: 0.008343154999977287
inference time for step 2: 0.008581660000004376
inference time for step 3: 0.008094579999976759
inference time for step 4: 0.008173222999971586
inference time for step 5: 0.008233745999973507
inference time for step 6: 0.008283402000017759
average loss: 2.965711763926915
total inference time: 0.07023502599992071
total inference time / #samples: 8.26294423528479e-05
Test Accuracy: 0.4835294117647059
Test Precision: 0.49162161205745625
Test Recall: 0.4835294117647059
Test F1-Score(weighted average): 0.4637483731367207
Test classification Report:
              precision    recall  f1-score   support

           c  0.4905660377 0.3035019455 0.3750000000       257
           e  0.5151515152 0.3730407524 0.4327272727       319
           n  0.4652173913 0.7810218978 0.58310626

In [12]:
for premise, hypothesis, true_label, predicted_label in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(premise, hypothesis, true_label, predicted_label))

او را نیز بکشتند و پس از او، هیچ‌کس را نیافتند که شایسته پادشاهی باشد.	او را نیز بکشتند و پس از او پسرش را شایسته ترین فرد برای پادشاهی یافتند.	c	n
نعمت‌زاده همچنین با اشاره به ارتقاء کیفیت خودروهای داخلی طی چند سال گذشته، گفت: با اقدامات صورت گرفته کیفیت خودروهای داخلی رو به بهبود است، اما تا وضع مطلوب هنوز فاصله داریم.	به گفته‌ی وی تولید خودرو در مقایسه با سال‌های گذشته افزایش یافته است.	n	e
صابر دین‌پژوه‌ رتبه اول علوم ریاضی و فنی کنکور سال 96 که از شهر تبریز حائز این رتبه شده بود در مورد موفقیت خود اظهار کرد: پیش از اعلام نتایج و با احتساب درصد‌های کنکور حدس می‌زدم که یکی از رتبه‌های برتر و تک رقمی کنکور را کسب کنم زیرا در کنکورهای‌ آزمایشی در طول سال تحصیلی همواره‌ رتبه‌های تک رقمی و برتر را کسب می‌کردم.	رتبه‌ی اول علوم ریاضی و فنی کنکور از نتیجه‌ی کنکور خود متعجب بود. 	c	c
وی پس از رسیدن به کامپیوتر اصلی وارد ماتریکس شده و نبرد سختی بین او و اسمیت رخ می‌دهد.	در این فیلم شبکه های اینترنتی که در آن زمان پدیدۀ جدیدی بود بخوبی تصویر شده است.	n	n
به‌طور کلی سلسه مراتب و تقسیم‌بندی حیا

In [13]:
output_file_name = "textual_entailment-parsinlu-natural_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for premise, hypothesis, true_label, predicted_label in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(premise, hypothesis, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### parsinlu mnli subset

In [14]:
test_mnli, test_labels = te_model.load_dataset_test_file(dataset_name="parsinlu-mnli", dataset_file="./parsinlu/data/entailment/merged_with_farstail/test_translation.tsv")

test part:
 #premise_hypothesis: 823, #label: 823


In [15]:
!nvidia-smi
!lscpu

Wed Aug  4 12:58:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    35W /  70W |   5936MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
evaluation_output = te_model.evaluation(test_mnli, test_labels, device, max_length=te_model.config.max_position_embeddings, batch_size=128)

label_count: {'n': 228, 'e': 291, 'c': 304}
#premise_hypothesis:823, #labels:823
#batch: 7
Start to evaluate test data ...
inference time for step 0: 0.026111711999988074
inference time for step 1: 0.00831574000000046
inference time for step 2: 0.009314844000016365
inference time for step 3: 0.00890121600002658
inference time for step 4: 0.009285278999982438
inference time for step 5: 0.009276614999976118
inference time for step 6: 0.008169957000006889
average loss: 3.4104264804295132
total inference time: 0.07937536299999692
total inference time / #samples: 9.6446370595379e-05
Test Accuracy: 0.3863912515188335
Test Precision: 0.44137432878734484
Test Recall: 0.3863912515188335
Test F1-Score(weighted average): 0.36973025283114413
Test classification Report:
              precision    recall  f1-score   support

           c  0.4913294798 0.2796052632 0.3563941300       304
           e  0.4832214765 0.2474226804 0.3272727273       291
           n  0.3213572854 0.7061403509 0.441700960

In [17]:
for premise, hypothesis, true_label, predicted_label in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(premise, hypothesis, true_label, predicted_label))

آنها به عنوان (الف) رسید جبران خسارت طبقه بندی می شوند (یعنی	چیز خوبی است که آنها در این دسته قرار می گیرند.	n	c
مورد دیگری که باید ببینید ، فیلم خوابیدن با دشمن است.	باید خوابیدن با دشمن را ببینی.	e	c
داستانهای افسانه ای ارزش خود را دارند ، اما پنهان کاری کثیف است.	پنهان کردن چیزها فقط کثیف است ، در حالی که در داستان افسانه ای، جلال وجود دارد	e	c
او یک دستش را به سمت سمندر آورد ، آن را به آرامی نوازش کرد و آن را دوباره روی سینه دیو گذاشت.	سمندر پذیرای حرکات دوستانه او بود.	n	n
بنابراین ، با دقت بسیار زیاد ، افشاگری نهایی فیلم Zapruder این است که ابراهیم Zapruder خود توطئه گر بود.	در فیلم Zapruder فاش می شود که او توطئه گر بود ، این برای عموم شوک آور بود.	n	n
ایده خوبی هست	بهترین ایده ای بود که من شنیده ام.	n	n
جان ، در حالی که هنوز هم به چشمان آدرین نگاه می کرد ، دید که یک تیغه بسیار تیز قفسه سینه کال را شکافت .	تیغی از سینه کال عبور کرد.	e	n
برای به دست آوردن یک تجربه‌ی کامل ، در غروب آفتاب یا خارج از فصل بازدید کنید.	بهتر است در یکشنبه آفتابی یا در فصل کم بازدید کنید.	e	e
می توانید 

In [18]:
output_file_name = "textual_entailment-parsinlu-mnli_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for premise, hypothesis, true_label, predicted_label in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(premise, hypothesis, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### parsinlu farstail subset

In [19]:
test_farstail, test_labels = te_model.load_dataset_test_file(dataset_name="parsinlu-farstail", dataset_file="./parsinlu/data/entailment/merged_with_farstail/test_farstail.tsv")

test part:
 #premise_hypothesis: 1564, #label: 1564


In [20]:
!nvidia-smi
!lscpu

Wed Aug  4 12:58:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    35W /  70W |   5936MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
evaluation_output = te_model.evaluation(test_farstail, test_labels, device, max_length=te_model.config.max_position_embeddings, batch_size=128)

label_count: {'c': 510, 'n': 535, 'e': 519}
#premise_hypothesis:1564, #labels:1564
#batch: 13
Start to evaluate test data ...
inference time for step 0: 0.027434343000038552
inference time for step 1: 0.007969625000043834
inference time for step 2: 0.00897014500003479
inference time for step 3: 0.008663694000006217
inference time for step 4: 0.008651259000032496
inference time for step 5: 0.00805952099995011
inference time for step 6: 0.00810705400004963
inference time for step 7: 0.008301576999997451
inference time for step 8: 0.008312074999992092
inference time for step 9: 0.008109922000016923
inference time for step 10: 0.009546391000014864
inference time for step 11: 0.009979793000013615
inference time for step 12: 0.00901556100001244
average loss: 2.152243889295138
total inference time: 0.131120960000203
total inference time / #samples: 8.383693094642137e-05
Test Accuracy: 0.6400255754475703
Test Precision: 0.653405975179831
Test Recall: 0.6400255754475703
Test F1-Score(weighted a

In [22]:
for premise, hypothesis, true_label, predicted_label in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(premise, hypothesis, true_label, predicted_label))

دوران امامت امام صادق علیه السلام، مصادف است با ترجمه آثار یونانی و گسترش مبارزات فکری و ایدئولوژیکی و نیز ظهور مذاهب و مکتب های انحرافی.	امام سجاد (ع) در دورانی امامت کردند که همزمان با ترجمه آثار یونانی، ظهور مذاهب و مکتب های انحرافی بود.	c	c
دوران امامت امام صادق علیه السلام، مصادف است با ترجمه آثار یونانی و گسترش مبارزات فکری و ایدئولوژیکی و نیز ظهور مذاهب و مکتب های انحرافی.	دستگاه فاسد حکومتی با صرف هزینه های هنگفت، سعی در جعل احادیث و ایجاد انحراف در مکتب تشیع کرده است.	n	n
با شهادت امام رضا(ع) مرحله جدیدی از تلاش ائمه آغاز شد که «دوران محنت اهل بیت» نام دارد.	دوران محنت اهل بیت پس از شهادت امام رضا(ع) آغاز گردید.	e	e
با شهادت امام رضا(ع) مرحله جدیدی از تلاش ائمه آغاز شد که «دوران محنت اهل بیت» نام دارد.	بعد از به شهادت رسیدن امام هادی(ع) دوران محنت اهل بیت شروع شد.	c	c
با شهادت امام رضا(ع) مرحله جدیدی از تلاش ائمه آغاز شد که «دوران محنت اهل بیت» نام دارد.	حضرت جواد(ع) در سال ۱۹۵ هجری در مدینه ولادت یافت.	n	n
توحید اَفعالی، باور به اینکه اراده واحدی بر جهان حاکم است. یعنی جز یک 

In [23]:
output_file_name = "textual_entailment-parsinlu-farstail_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for premise, hypothesis, true_label, predicted_label in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(premise, hypothesis, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()